In [5]:
#!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [8]:
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [9]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [10]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [11]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [12]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [13]:
y_pred.mean()

16.191691679979066

In [18]:
year=2021
month =2

In [19]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [21]:
df.ride_id[:3]

1    2021/02_1
2    2021/02_2
3    2021/02_3
Name: ride_id, dtype: object

In [22]:
df_result = pd.DataFrame()

In [23]:
df_result['ride_id'] = df['ride_id']

In [24]:
df_result['predictions'] = pd.Series(y_pred)

In [25]:
df_result.head(2)

,ride_id,predictions
1,2021/02_1,13.740422
2,2021/02_2,15.593339


In [27]:
output_file= "./outputs"

In [30]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [33]:
#ls -lh

total 38384
-rw-r--r--@ 1 subramanianhariharan  staff    17K Jun 26 22:10 model.bin
-rw-r--r--  1 subramanianhariharan  staff    19M Jun 26 22:35 outputs
-rw-r--r--  1 subramanianhariharan  staff     0B Jun 26 22:01 starter.ipynb
